In [1]:
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers import Input, Conv2D

import numpy as np
import random
from audiomentations import AddBackgroundNoise
import pandas as pd
from tqdm import tqdm
from keras_tqdm import TQDMNotebookCallback
import keras
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbEvalCallback


2024-11-25 14:45:54.757742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732513554.768989 2947932 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732513554.772488 2947932 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 14:45:54.783918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
class CONFIG:
    SEED = 42
    CLASSIFIER_LR = 1e-4
    EPOCH = 200
    BATCH_SIZE = 8
    VOICE_DIR = './cv-corpus-19.0-2024-09-13/ko/clips/'
    NOISE_DIR = './ESC-50-master/audio/'

In [5]:
wandb.init(project="denoiser")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: is-jang (is-jang-pusan-national-university). Use `wandb login --relogin` to force relogin


In [6]:
def periodic_hann_window(window_length, dtype):
    return 0.5 - 0.5 * tf.math.cos(2.0 *
                                   np.pi *
                                   tf.range(tf.cast(window_length, tf.float32)) /
                                   tf.cast(window_length, tf.float32))

In [7]:
def wave2log_mel_spectrogram(wave):
    signal_stft = tf.signal.stft(tf.cast(wave, tf.float32),
                                 frame_length=400,
                                 frame_step=100,
                                 fft_length=1024,
                                 window_fn=periodic_hann_window)
    spectogram = tf.abs(signal_stft)
    spectogram = spectogram[:, :-1]
    # print(spectogram.shape)

    # linear_to_mel = tf.signal.linear_to_mel_weight_matrix(40,
    #                             signal_stft.shape[-1],
    #                             16000,
    #                             300.0,
    #                             4000.0)
    # mel_spectrogram = tf.tensordot(spectogram, linear_to_mel, 1)
    # log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-12)
    return spectogram

In [8]:
def data_generator(df):
    for _, row in df.iterrows():
        aug_path = row['aug']
        gt_path = row['gt']

        aug_audio_raw = tf.io.read_file(aug_path)
        aug_wave, sr = tf.audio.decode_wav(aug_audio_raw, desired_channels=1)
        aug_wave = tf.squeeze(aug_wave, axis=-1)
        # print(wave.shape) 
        aug_spectrogram = wave2log_mel_spectrogram(aug_wave)
        aug_spectrogram = np.expand_dims(aug_spectrogram, axis=-1)
        # print(log_mel_spectrogram.shape) #(49, 80 ,1)

        gt_audio_raw = tf.io.read_file(gt_path)
        gt_wave, sr = tf.audio.decode_wav(gt_audio_raw, desired_channels=1)
        gt_wave = tf.squeeze(gt_wave, axis=-1)
        # print(wave.shape) 
        gt_spectrogram = wave2log_mel_spectrogram(gt_wave)
        gt_spectrogram = np.expand_dims(gt_spectrogram, axis=-1)
        # print(log_mel_spectrogram.shape) #(49, 80 ,1)
        

        yield aug_spectrogram, gt_spectrogram


In [9]:
train_df = pd.read_csv('train_denoiser_dataset.csv')


In [10]:
valid_df = pd.read_csv('valid_denoiser_dataset.csv')


In [11]:
test_df = pd.read_csv('test_denoiser_dataset.csv')


In [12]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_df),
    output_signature=(
        tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float32)
    )
).batch(CONFIG.BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(valid_df),
    output_signature=(
        tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float32)
    )
).batch(CONFIG.BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_df),
    output_signature=(
        tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(512, 512, 1), dtype=tf.float32)
    )
).batch(CONFIG.BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

I0000 00:00:1732513559.437165 2947932 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 115 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5
2024-11-25 14:45:59.447530: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 115.06MiB (120651776 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-11-25 14:45:59.447592: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 103.56MiB (108586752 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-11-25 14:45:59.447644: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 93.20MiB (97728256 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-11-25 14:45:59.447693: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to alloca

In [13]:

# Define the encoder-decoder model with skip connections
input_layer = layers.Input(shape=(512, 512, 1))

# Encoder
conv1 = layers.Conv2D(3, kernel_size=3, padding='same', activation='relu')(input_layer)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(8, kernel_size=3, padding='same', activation='relu')(pool1)
pool2 = layers.MaxPooling2D((2, 2))(conv2)

# Decoder
upconv1 = layers.Conv2DTranspose(16, kernel_size=2, strides=2, activation='relu', padding='same')(pool2)
skip1 = layers.Concatenate()([upconv1, conv2])  # Skip connection from conv2

upconv2 = layers.Conv2DTranspose(8, kernel_size=2, strides=2, activation='relu', padding='same')(skip1)
skip2 = layers.Concatenate()([upconv2, conv1])  # Skip connection from conv1

# Output layer
output_layer = layers.Conv2D(1, kernel_size=3, padding='same', activation='sigmoid')(skip2)

# Create the model
model = models.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 512,  │         30 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 256,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │        224 │ max_pooling2d[0]… │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 256, 256,  │        528 │ max_pooling2d_1[… │
│ (Conv2DTranspose)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 24)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 512, 512,  │        776 │ concatenate[0][0] │
│ (Conv2DTranspose)   │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512, 512,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 11)               │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 512, 512,  │        100 │ concatenate_1[0]… │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,658 (6.48 KB)

 Trainable params: 1,658 (6.48 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
class WandbClfEvalCallback(WandbEvalCallback):
    def __init__(self, validation_data, data_table_columns, pred_table_columns, num_samples=100):
        """
        Callback to log augmented (aug) and ground truth (gt) images during training.
        
        Args:
            validation_data (tuple): A tuple containing validation images and ground truth labels.
            data_table_columns (list): Column names for the ground truth table.
            pred_table_columns (list): Column names for the predictions table.
        """
        super().__init__(data_table_columns, pred_table_columns)
        self.validation_data = validation_data
        self.data_table = wandb.Table(columns=data_table_columns)
        self.pred_table = wandb.Table(columns=pred_table_columns)


    def add_ground_truth(self, logs=None):
        for idx, (aug, gt) in enumerate(self.validation_data):
            self.data_table.add_data(
                idx,
                wandb.Image(aug[0].numpy(), caption=f"Augmented {idx}"),
                wandb.Image(gt[0].numpy(), caption=f"Ground Truth {idx}")
            )
    def add_model_predictions(self, epoch, logs=None):
        for idx, (aug_spectrogram, _) in enumerate(self.validation_data.take(self.num_samples)):
            pred_spectrogram = self.model.predict(tf.expand_dims(aug_spectrogram, axis=0), verbose=0)
            self.pred_table.add_data(
                epoch,
                idx,
                wandb.Image(aug_spectrogram[0].numpy(), caption=f"Augmented {idx}"),
                wandb.Image(pred_spectrogram[0], caption=f"Prediction {idx}")
            )

In [15]:

model.compile(optimizer=Adam(learning_rate=CONFIG.CLASSIFIER_LR),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, restore_best_weights=True)


history = model.fit(train_dataset, 
                    verbose=2, 
                    epochs=CONFIG.EPOCH, 
                    validation_data=valid_dataset, 
                    callbacks=[early_stopping, 
                        WandbMetricsLogger(), 
                        WandbClfEvalCallback(validation_data=valid_dataset, 
                        data_table_columns=["idx", "aug", "gt"],
                        pred_table_columns=["epoch", "idx", "aug", "gt", "pred"])], 
                    steps_per_epoch=len(train_df)//CONFIG.BATCH_SIZE, validation_steps=len(valid_df)//CONFIG.BATCH_SIZE)

2024-11-25 14:46:00.625718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:255] Failed to make cuFFT batched plan: 5
2024-11-25 14:46:00.625737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:322] Initialize Params: rank: 1 elem_count: 1024 input_embed: 1024 input_stride: 1 input_distance: 1024 output_embed: 513 output_stride: 1 output_distance: 513 batch_count: 512
2024-11-25 14:46:00.625743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:331] Failed to initialize batched cufft plan with customized allocator: Failed to make cuFFT batched plan.
2024-11-25 14:46:00.625757: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INTERNAL: Failed to create cuFFT batched plan with scratch allocator
2024-11-25 14:46:00.626841: W tensorflow/core/framework/op_kernel.cc:1829] UNKNOWN: InternalError: {{function_node __wrapped__RFFT_device_/job:localhost/replica:0/task:0/device:GPU:0}} Failed to create cuFFT batched plan wit

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} InternalError: {{function_node __wrapped__RFFT_device_/job:localhost/replica:0/task:0/device:GPU:0}} Failed to create cuFFT batched plan with scratch allocator [Op:RFFT] name: 
Traceback (most recent call last):

  File "/media/insung/새 볼륨/gamba/vad/myenv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/media/insung/새 볼륨/gamba/vad/myenv/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/media/insung/새 볼륨/gamba/vad/myenv/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_2947932/1456906717.py", line 10, in data_generator
    aug_spectrogram = wave2log_mel_spectrogram(a

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

In [ ]:
# model.trainable = False
# tf.saved_model.save(model, "./denoiser2")
# # model.summary()

In [ ]:
# model.save('./denoiser2.h5')


In [ ]:
# model = tf.keras.models.load_model('./denoiser2.h5')
# model.summary()